# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

if not os.path.exists("rice.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/rice.png --no-check-certificate
rice = cv2.imread('rice.png', cv2.IMREAD_GRAYSCALE)
plt.gray()
plt.imshow(rice)
plt.xticks([])
plt.yticks([])
plt.title("rice.png")
plt.show()
def adaptive_without_inte(img,k):
    W = 2**k
    imgV2 = np.zeros(img.shape)
    for i in range(0,256,W):
        for j in range(0,256,W):
            imgV2[i:i+W,j:j+W] = img[i:i+W,j:j+W].mean() < img[i:i+W,j:j+W]
    return imgV2
k = 4
plt.imshow(adaptive_without_inte(rice,k))
plt.xticks([])
plt.yticks([])
plt.title("rice.png adaptive binaryzation\n without interpolation")
plt.show()




4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego okna wyliczany jest na podstawie progów z sąsiednich okien.
   
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:
def adaptive_with_inte(img,k):
    W = 2**k
    imgV2 = np.zeros(img.shape)
    tab = [[]]
    for i in range(0,256,W):
        for j in range(0,256,W):
            tab[i//W].append(img[i:i+W,j:j+W].mean())
        tab.append([])
    tab.pop(-1)
    tab = np.array(tab,dtype='uint8')
    
    ## rogi obrazu
    for i in range(img.shape[0]):
        for j in range(img.shape[0]):
            jT = j//W
            iT = i//W
            if ((i > img.shape[0] - W//2 - 1) and (j > img.shape[0] - W//2 - 1)) or ((i < W//2 + 1) and (j < W//2 + 1)) or ((i > img.shape[0] - 1 - W//2) and (j < W//2 + 1)) or ((i < W//2 + 1) and (j > img.shape[0] - 1 - W//2)):
                imgV2[i,j] = tab[iT,jT] < img[i,j]
            elif (j > img.shape[0] - W//2 - 1) or ((j < W//2 + 1)):
                ## pion
                ## miejsza wsp
                iT = (i-W//2)//W
                dY1 = i - W/2 - iT*W
                dY2 = (iT+1)*W - (i - W/2) 
                imgV2[i,j] = float(float(dY1+1)/float(W)*tab[iT,jT]+float(dY2)/float(W)*tab[iT,jT]) < img[i,j]
            elif (i > img.shape[0] - 1 - W//2) or (i < W//2 + 1):
                ## poziom
                ## miejsza wsp
                jT = (j-W//2)//W
                dX1 = j - W/2 - jT*W
                dX2 = (jT+1)*W - (j - W/2)  
                imgV2[i,j] = float(float(dX1)/float(W)*tab[iT,jT+1]+float(dX2)/float(W)*tab[iT,jT]) < img[i,j]
            else:
                iT = (i-W//2)//W
                dY1 = i - W/2 - iT*W
                dY2 = (iT+1)*W - (i - W/2)  
                jT = (j-W//2)//W
                dX1 = j - W/2 - jT*W
                dX2 = (jT+1)*W - (j - W/2)  
                imgV2[i,j] = float(float(dX1)/float(W)*float(dY1)/float(W)*tab[iT+1,jT+1]+float(dX2)/float(W)*float(dY2)/float(W)*tab[iT,jT] + float(dX1)/float(W)*float(dY2)/float(W)*tab[iT,jT+1]+float(dX2)/float(W)*float(dY1)/float(W)*tab[iT+1,jT]) < img[i,j]
                pass
    ## ramka

    return imgV2
k = 4
plt.imshow(adaptive_with_inte(rice,k))
plt.xticks([])
plt.yticks([])
plt.title("rice.png adaptive binaryzation\n with interpolation")
plt.show()


In [ ]:
def binaryzation(img, threshold):
    bin_img = (threshold < img).astype('uint8')
    return bin_img
def Otsu(img):
    img_hist = cv2.calcHist([img], [0], None, [256], [0,256])/(img.shape[0]*img.shape[1])
    img_hist = np.squeeze(img_hist)
    img_hist_cum_norm = img_hist.cumsum()
    img_mean = img.mean()
    pi = img_hist  #np.arange(256)
    sigma_B = np.zeros(256)
    m = 0.0
    k = 0
    for i in range(256):
        m += i*pi[i]
        if 0<img_hist_cum_norm[i]<1:
            sigma_B[i] = (img_mean*img_hist_cum_norm[i] - m)**2/(img_hist_cum_norm[i]*(1-img_hist_cum_norm[i]))
        if sigma_B[k] < sigma_B[i]:
            k = i
    return k
def automatic_binaryzacion_otsu(img):
    k = Otsu(img)
    return binaryzation(img,k)

def local_binaryzation(img,W):
    half = (W-1)//2
    half_2 = (W-1) - half
    res = np.array([[ img[i-half:i+half_2,j-half:j+half_2].mean()<img[i,j]  for j in range(half,img.shape[1]-half_2)]for i in range(half,img.shape[0]-half_2)])
    return res

def local_binaryzation_std(img,W,R,k):
    half = (W-1)//2
    half_2 = (W-1) - half
    res = np.array([[ (img[i-half:i+half_2,j-half:j+half_2].mean()*(1-x) if (x:=k*((img[i-half:i+half_2,j-half:j+half_2].std()/R)-1)) < 0 else img[i-half:i+half_2,j-half:j+half_2].mean()*(1+x))<img[i,j]  for j in range(half,img.shape[1]-half_2)]for i in range(half,img.shape[0]-half_2)])
    return res
W = 2**k

fig,ax = plt.subplots(3,2)
fig.set_size_inches(16,24)
ax[0,0].imshow(rice)
ax[0,0].set_title("orginal picture")
ax[0,0].set_xticks([])
ax[0,0].set_yticks([])
ax[0,1].imshow(automatic_binaryzacion_otsu(rice))#
ax[0,1].set_title("Otsu binaryzation")
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
ax[1,0].imshow(local_binaryzation(rice,W))
ax[1,0].set_title("local mean binaryzation")
ax[1,0].set_xticks([])
ax[1,0].set_yticks([])
ax[1,1].imshow(local_binaryzation_std(rice,W, 128, 0.2))
ax[1,1].set_title("Sauvola's binaryzation")
ax[1,1].set_xticks([])
ax[1,1].set_yticks([])
ax[2,0].imshow(adaptive_without_inte(rice,k))
ax[2,0].set_title("adaptive binaryzation without interpolation")
ax[2,0].set_xticks([])
ax[2,0].set_yticks([])
ax[2,1].imshow(adaptive_with_inte(rice,k))
ax[2,1].set_title("adaptive binaryzation with interpolation")
ax[2,1].set_xticks([])
ax[2,1].set_yticks([])
plt.suptitle("rice.png binaryzation techniques")
plt.show()

widać jednoznacznie że najlepszymi formami binaryzacji jest lokalna binaryzacja Sauvoli oraz lokalna binaryzacja na podstawie średniej lecz są one bardzo czasochłonne gdyż dla każdego piksela z osobna należy policzyć dla nich średnią lub odchylenie standardowe otoczenia. w przypadku metody otsu nie ma problemu z dużymi obliczeniami gdyż próg binaryzacji liczy się tylko raz. Poważnym problemem metody Otsu jest brak odporności na zmienne oświetlenie z uwagi na jednakowy próg binaryzacji obiekty w skrajnych odcieniach mogą zlać się z tłem. lokalna binaryzacja w oknach jest jedynym z lepszych rozwiązań które łączą szybkość obliczeń z dokładnością z uwagi na zastosowanie interpolacji znika problem powstawania dziwnych artefaktów na skrajach okien
